In [1]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')

# Launch DPS for extract_filter_atl08.py
using the rebinned 30m h5 granules

In [2]:
from os import path
import os, glob

In [3]:
curr_dir = wk_dir = os.path.dirname(os.path.realpath('__file__'))
print(curr_dir)

/projects/icesat2_boreal/dps/alg_2-3


## Test extract_filter_atl08.py code on Test Data
We are running extract_filter_atl08.py (but *FILTERING* is turned off):

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python /projects/icesat2_boreal/lib/extract_filter_atl08.py -o "/projects/test_data/test_data_30m" -i "https://maap-ops-workspace.s3.amazonaws.com.com/lduncanson/dps_output/run_rebinning_ubuntu/master/2022/02/25/18/51/17/542274/ATL08_30m_20190721220156_03640403_005_01.h5" --no-filter-qual --do_30m

python lib/extract_filter_atl08.py -i "path/to/h5file" -o "path/of/out/dir" --no-filter-qual --do_30m

### Build the input list of granules

In [5]:
%%time
INPUT_ATL08_GRANULE_LIST = glob.glob("/projects/my-private-bucket/dps_output/run_rebinning_ubuntu/master/2022/02/**/*.h5", recursive=True)
len(INPUT_ATL08_GRANULE_LIST)

CPU times: user 192 ms, sys: 315 ms, total: 508 ms
Wall time: 1min 31s


1019

## Run a DPS job across the list
glob.glob takes quite awhile to list all the files in the output directory of the 30 m outputs; divide this up by processing date to loop over a job submission per file, with a file set per day of 30 m outputs


In [ ]:
RUN_DPS  = True

if RUN_DPS:
    ##################################
    #Test DPS submission on a single file
    for i, INPUT_ATL08_GRANULE in enumerate(INPUT_ATL08_GRANULE_LIST[0]):
        
        DPS_num = i+1
        in_param_dict = {
                            'input_file': INPUT_ATL08_GRANULE
                        }

        submit_result = maap.submitJob(
                identifier='run_extract_filter_atl08',
                algo_id='run_extract_filter_atl08_ubuntu',
                version='master',
                username='lduncanson', # username needs to be the same as whoever created the workspace
                queue='maap-dps-worker-8gb',
                **in_param_dict
            )

        #submit_result = 'submit test'
        if DPS_num in [1, 100, 500, 1000, 3000, len(INPUT_ATL08_GRANULE_LIST)]:
           print(f"DPS run num: {DPS_num}, granule name: {INPUT_ATL08_GRANULE}, job info: {submit_result}") 
 